# Figure DGVM Deficit Ecosystem Barplot

In [ ]:
# Libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

In [ ]:
# Directories
dir05x = '../paper_deficit/output/05_prep_other/fig_dgvm/'
dir06 = '../paper_deficit/output/06_eval/'

---

In [ ]:
# List of DGVMs
list_dgvm = ['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 
             'jules', 'lpjguess', 'lpjwsl', 'orchidee']

In [ ]:
# Get data
df_g = pd.read_csv(os.path.join(dir05x, 'data_dgvm_global.csv'))
df_e = pd.read_csv(os.path.join(dir05x, 'data_dgvm_ecozone.csv'))

# Create individual dataframes for each data source
df_pot = df_g[df_g.model == 'ganzenmueller'].drop('model', axis=1)
df_erb = df_g[df_g.model == 'erb'].drop('model', axis=1)
df_mo = df_g[df_g.model == 'mo'].drop('model', axis=1)
df_walker = df_g[df_g.model == 'walker'].drop('model', axis=1)
df_sand030 = df_g[df_g.model == 'sanderman030'].drop('model', axis=1)
df_sand100 = df_g[df_g.model == 'sanderman100'].drop('model', axis=1)
df_sand200 = df_g[df_g.model == 'sanderman200'].drop('model', axis=1)
df_dgvm_m = df_g[df_g.model == 'dgvm_mean'].drop('model', axis=1)
df_dgvm = df_g[df_g.model.isin(list_dgvm)]

# Merge df_g and df_e
df_all = pd.concat([df_e, df_g.assign(cat='global')])

In [ ]:
# colors for vegetation and soil carbon
# selected with https://color.adobe.com/ and https://projects.susielu.com/viz-palette
c_cveg, c_soc = '#73bd83', '#8f6358' 
c_edge = '#000000'

# x-axis order
list_xaxis = list_dgvm
# x-axis labels
list_xlabels = ['CABLE-POP', 'CLASSIC', 'CLM', 'DLEM', 'IBIS', 'ISAM', 'JSBACH',
                'JULES', 'LPJ-GUESS', 'LPJ-WSL', 'ORCHIDEE']

In [ ]:
# Plot
fig, axs = plt.subplots(figsize=(9,4.5), ncols=3, nrows=2, sharex=True, dpi=600)
axs = axs.ravel()

for i in range(0, 6):
    cat = ['global', 'trop1', 'trop2', 'subt', 'temp', 'bore'][i]  # polar not shown
    subtitle = ['Global', 'Tropical Rainforest', 'Other Tropical', 
                'Subtropical', 'Temperate', 'Boreal'][i]
        
    df_sel = df_all[(df_all.cat == cat)] \
        .set_index('model') \
        .reindex(list_xaxis) \
        .reset_index() \
        .assign(xlabels=list_xlabels)

    df_sel_mean = df_all[(df_all.cat == cat) & (df_all.model == 'dgvm_mean')]
    df_sel_ganz = df_all[(df_all.cat == cat) & (df_all.model == 'ganzenmueller')]    

    axs[i].axvspan(-0.5, 0.5, facecolor='#bdbdbd', alpha=0.5)
    axs[i].axvspan(1.5, 2.5, facecolor='#bdbdbd', alpha=0.5)
    axs[i].axvspan(3.5, 4.5, facecolor='#bdbdbd', alpha=0.5)
    axs[i].axvspan(5.5, 6.5, facecolor='#bdbdbd', alpha=0.5)
    axs[i].axvspan(7.5, 8.5, facecolor='#bdbdbd', alpha=0.5)    
    axs[i].axvspan(9.5, 10.5, facecolor='#bdbdbd', alpha=0.5)

    axs[i].bar(df_sel.index-0.2, df_sel.cveg_d * -1, width=0.4, 
               color=c_cveg, edgecolor=c_edge, zorder=2)
    axs[i].bar(df_sel.index+0.2, df_sel.csoil_d * -1, width=0.4, 
               color=c_soc, edgecolor=c_edge, zorder=2)

    axs[i].axhline(df_sel_mean.cveg_d.item() * -1, color=c_cveg, ls='--', zorder=1)
    axs[i].axhline(df_sel_mean.csoil_d.item() * -1, color=c_soc, ls='--', zorder=1)
    axs[i].axhline(df_sel_ganz.cveg_d.item() * -1, color=c_cveg, ls=':', zorder=1)
    axs[i].axhline(df_sel_ganz.csoil_d.item() * -1, color=c_soc, ls=':', zorder=1)
    
    axs[i].set_title(subtitle)
    
    axs[i].set_xticks(df_sel.index)
    axs[i].set_xticklabels(df_sel.xlabels, rotation=90, ha='center')
    
    axs[i].set_xlim(-0.6, 10.5)       
    axs[i].hlines(0, -1, 16, colors='#000000', linewidth=2)   
    axs[i].tick_params(left=True, bottom=False)
    
for ax in axs:
    linewidth = 2
    ax.spines['right'].set_visible(False)  # remove right plot boundary
    ax.spines['top'].set_visible(False)  # remove top plot boundary
    ax.spines['left'].set_linewidth(linewidth)  # make left axis thicker
    ax.spines['bottom'].set_linewidth(linewidth)  # make bottom axis thicker
    ax.xaxis.set_tick_params(width=linewidth)  # make x-axis tick marks thicker
    ax.yaxis.set_tick_params(width=linewidth)  # make y-axis tick marks thicker

fig.supylabel('Carbon deficit (PgC)')

# Define legend elements
legend_elements = [
    Patch(facecolor=c_cveg, edgecolor=c_edge, label='AGBC + BGBC'),
    Patch(facecolor=c_soc, edgecolor=c_edge, label='SOC'),
    Line2D([0], [0], color=c_cveg, linestyle='--', label='DGVM mean'),
    Line2D([0], [0], color=c_cveg, linestyle=':', label='This study')
]

# Add legend below the plot
fig.legend(handles=legend_elements, loc='lower center', ncol=4, frameon=False, bbox_to_anchor=(0.5, -0.08))

plt.tight_layout()

# Export
fstr_out = 'figs06_dgvm_deficit_ecosystem_bar'
plt.savefig(os.path.join(dir06, f'pdf/{fstr_out}.pdf'), bbox_inches='tight', dpi=600)
plt.savefig(os.path.join(dir06, f'png/{fstr_out}.png'), bbox_inches='tight', dpi=600);